# Showquester

In [1]:
# load libraries
import os
import spotipy
import spotipy.util as util

import pickle

import requests
import pandas as pd

# show all the things
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import datetime

# load environmental variables
from dotenv import load_dotenv
load_dotenv()

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from math import ceil as round_up

In [149]:
venue_info = pd.read_csv('venues.csv')
venue_info

,venue,url
0,McMenamin's Crystal Ballroom,https://www.mcmenamins.com/'
1,Mississippi Studios,https://www.mississippistudios.com/
2,Polaris Hall,https://www.mississippistudios.com/
3,Doug Fir Lounge,https://www.dougfirlounge.com/
4,Wonder Ballroom,https://wonderballroom.com/
5,Holocene,http://www.holocene.org/
6,Roseland Theater,http://roselandpdx.com/


In [287]:
# change these values for each venue
venue_name = venue_info.venue[2]
venue_url = venue_info.url[2]

In [288]:
#### FUNCTIONS

# search for venue and retrieve id
# NOTE: venue name must exactly match for the search to be successful
def get_venue_id(venue_name): 
    req = f'https://api.songkick.com/api/3.0/search/venues.json?query={venue_name}&apikey={sk_api_key}'
    response = requests.get(req)
    num_responses = response.json()['resultsPage']['totalEntries']
    if num_responses > 0:
        results = response.json()['resultsPage']['results']
        top_hit = results['venue'][0]
        venue_id = top_hit['id']
        displayName = top_hit['displayName']
        print(f'Found: {displayName}')
        return venue_id
    else:
        print('No venue found with that name.')

# get a venue's upcoming events from songkick
def get_venue_events(venue_id):
    req = f'https://api.songkick.com/api/3.0/venues/{venue_id}/calendar.json?apikey={sk_api_key}'
    response = requests.get(req)
    # get number of events per page
    per_page = response.json()['resultsPage']['perPage']
    # get total number of events
    total_entries = response.json()['resultsPage']['totalEntries']
    # if multiple pages of events
    if total_entries > per_page:
        # calculate number of pages
        pages = round_up(total_entries/per_page)
        # save first page of results to venue_events
        venue_events = response.json()['resultsPage']['results']['event']
        # get subsequent pages of results
        for page_num in range(2, pages+1):
            # request next page of events
            req = f'https://api.songkick.com/api/3.0/venues/{venue_id}/calendar.json?apikey={sk_api_key}&page={page_num}'
            response = requests.get(req)
            page_events = response.json()['resultsPage']['results']['event']
            # add to venue_events list
            venue_events.extend(page_events)
    else:
        venue_events = response.json()['resultsPage']['results']['event']

    return venue_events


# create a dataframe of relevant data
def events_df(event_list):
    dates = []
    artists = []
    ids = []
    for event in event_list:

        # get performers
        performance = event['performance']
        num_performers = len(performance)
        # add to artists list
        for artist in performance:
            artists.append(artist['displayName'])
            ids.append(artist['id'])

        # get date
        event_date = event['start']['date']
        # add to dates list, once for each performer
        dates.extend([event_date] * num_performers)
        
    return pd.DataFrame(data={'artist':artists, 'date':dates, 'artist_id':ids})

def get_artist(search_str):
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False
        results = sp.search(search_str,type='artist')
    
    if results['artists']['total'] > 0:
        items = results['artists']['items']
        hits = {}
        for artist in items:
            hits[artist['uri']] = artist['name']

        best_hit = process.extractOne(search_str, hits)
        best_hit_uri = best_hit[2]
        artist_obj = sp.artist(best_hit_uri)
        return artist_obj
    else:
        print(f'\"{search_str}\" was not found on Spotify.')
        return None
        
def get_top_track(artist_uri):
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False
        results = sp.artist_top_tracks(artist_uri)
    
    top_tracks = results['tracks']
    top_track = []
    if top_tracks:
        for track in top_tracks:
            album_artist = track['album']['artists'][0]['uri']
            if album_artist == artist_uri:
                top_track = track['uri']
                return top_track
                break
            else:
                continue
        if not top_track:
            top_track = top_tracks[0]['uri']
            print(f'check {artist_uri} for top track {top_track}')
            return top_track
    else:
        artist_name = sp.artist(artist_uri)['name']
        print(f'*********No tracks found for {artist_name}*********')

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def update_description(venue_name, venue_url):
    todays_date = datetime.date.today()
    playlist_name = 'ShowQuester: ' + venue_name
    descr = f'A programmatically-generated playlist featuring artists coming soon to {venue_name}. Ordered by date, with this week up top and events farther in the future at the bottom. Updated {todays_date}. Go to {venue_url} for tickets. Check out my GitHub for details on how this playlist is generated: [COMING SOON]'
    results = sp.user_playlist_change_details(
            username, playlist_id=playlist_id, name=playlist_name, description=descr)
    print(f'Updated {playlist_name}')

## Get calendar from Songkick API

In [289]:
sk_api_key = os.environ.get('SONGKICK_API_KEY')
venue_id = get_venue_id(venue_name)
event_list = get_venue_events(venue_id)
shows = events_df(event_list)

Found: Polaris Hall


In [265]:
# pickle dataframe
today = datetime.date.today()
today = today.strftime('%m-%d-%Y')
pickle_name = today + "_" + venue_name + '_shows.pickle'
shows.to_pickle(pickle_name)

## Remove exact duplicates

In [290]:
# remove exact duplicates
artist_list = list(shows.artist)
artist_list = list(process.dedupe(artist_list, threshold=99, scorer=fuzz.token_sort_ratio))

## Connect to Spotify

In [299]:
# set API authorization vars and scope
client_id = os.environ.get('SPOTIPY_CLIENT_ID')
client_secret = os.environ.get('SPOTIPY_CLIENT_SECRET')
username = os.environ.get('SPOTIPY_USERNAME')
scope = 'user-library-read playlist-modify-private playlist-modify-public playlist-read-private'

# authenticate
token = util.prompt_for_user_token(
        username=username,
        scope=scope,
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri='http://localhost/')


sp = spotipy.Spotify(auth=token)

## Get SoundQuester playlist info, or create a new SoundQuester playlist

In [300]:
# get playlist names and uris
my_playlists = {}

if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace = False
    result = sp.user_playlists(username)

for each in result['items']:
    my_playlists[each['name']] = each['uri']

# search playlists for a SoundQuester venue playlist
results = [(name, uri) for name, uri in my_playlists.items() if venue_name in name]
# if venue playlist found, get uri
if results:
    playlist_name, playlist_uri = results[0]
    print(f'Found playlist for {venue_name} named "{playlist_name}"')
# if venue playlist missing, create new SoundQuester playlist
else:
    print(f'No playlist found for "{venue_name}"')
    playlist_name = 'SoundQuester: ' + venue_name
    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False
        results = sp.user_playlist_create(username, playlist_name, public=True)
        playlist_uri = results['uri']
        print(f'Created playlist for venue_name named "{playlist_name}"')
        print(playlist_uri)

Found playlist for Polaris Hall named "ShowQuester: Polaris Hall"


In [301]:
my_playlists

{'ShowQuester: Mississippi Studios': 'spotify:playlist:1ndO4967sqMKFsMapRUVXe',
 'ShowQuester: Polaris Hall': 'spotify:playlist:3wyoGUeXeFAkz048JxbeBV',
 'ShowQuester: Doug Fir Lounge': 'spotify:playlist:4TI0yIKfLQchbbUKs7c54r',
 'ShowQuester: Wonder Ballroom': 'spotify:playlist:5s5WCIvbu6Xexnj9kKwzjq',
 "ShowQuester: McMenamin's Crystal Ballroom": 'spotify:playlist:5seIWANnuh0K9y7YO6s4dN',
 'ShowQuester: Holocene': 'spotify:playlist:1j4XEP3eh73yaiEUBwmNm9',
 'ShowQuester: Roseland Theater': 'spotify:playlist:453jgCdKEvu4WTVtsUUlNu',
 'The Shape Of Punk To Come': 'spotify:playlist:1tEFRg4TCE9v1KVP8wIjv1',
 'Metal Essentials': 'spotify:playlist:37i9dQZF1DWWOaP4H0w5b0',
 'Post Punk 2k': 'spotify:playlist:37i9dQZF1DWYwMzXER4RFF',
 'Vaporwave': 'spotify:playlist:17HYiAIcwlDEg5RgVkm4L7',
 'album scrape': 'spotify:playlist:5vFdoBU8LfgtyV6it0G19Q',
 'modge podge 2': 'spotify:playlist:4C21taFJ1cZp963gHwxYSB',
 'The Dirty Thirty Get Down': 'spotify:playlist:3wtzpabS8oPQKqT71L6nxI',
 'throwback 

## Get Spotify artist objects for each performer

In [293]:
# retrieve all artist objects for performing artists
artist_obj = []
for artist in artist_list:
    artist_obj.append(get_artist(artist))

"The Shivas (OR)" was not found on Spotify.


## Pulling tracks to add to the playlist

In [294]:
tracks_to_add = []
for artist in artist_obj:
    if artist is not None:
        artist_uri = artist['uri']
        track = get_top_track(artist_uri)
        tracks_to_add.append(track)

check spotify:artist:3trCmV1zHVJj8OnEdzOrlD for top track spotify:track:0qOMTezsihwX8uhjE76HgG


## Update playlist with new track list

In [295]:
# filter out empty strings where no track was found
tracks_to_add = list(filter(None, tracks_to_add))

In [296]:
# batch tracks into 100's to respect Spotify Web API limits
track_batches = list(chunks(tracks_to_add, 100))

In [297]:
# update playlist with new track list
playlist_id = playlist_uri.split(':')[2]

for i in range(0,len(track_batches)):  # for each batch of tracks
    track_uris = track_batches[i]
    # if first batch, replace playlist 
    if i == 0:
        if token:
            sp = spotipy.Spotify(auth=token)
            sp.trace = False
            result = sp.user_playlist_replace_tracks(username, playlist_id, track_uris)
    else:
        if token:
            sp = spotipy.Spotify(auth=token)
            sp.trace = False
            results = sp.user_playlist_add_tracks(username, playlist_id, track_uris)

## Update playlist info

In [298]:
# update playlist info
update_description(venue_name, venue_url)

Updated ShowQuester: Polaris Hall
